# Scenario Studio – Fuel Bus Demo
Run PF with explicit fuel buses. Adjust the YAMLs to change topology/params.


In [1]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# 🔥 EnerGIS Scenario Studio\n",
    "Interaktive Optimierung von industriellen Energiesystemen mit expliziten Fuel Buses.\n",
    "\n",
    "## Quick Start\n",
    "1. Alle Zellen nacheinander ausführen (Shift+Enter)\n",
    "2. Ergebnisse werden automatisch nach `exports/` geschrieben\n",
    "3. Plots zeigen wichtigste KPIs direkt im Notebook"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Setup & Imports\n",
    "import sys\n",
    "import os\n",
    "from pathlib import Path\n",
    "\n",
    "# Sicherstellen dass energis importiert werden kann\n",
    "project_root = Path().absolute()\n",
    "if str(project_root) not in sys.path:\n",
    "    sys.path.insert(0, str(project_root))\n",
    "\n",
    "import pandas as pd\n",
    "import matplotlib.pyplot as plt\n",
    "import warnings\n",
    "warnings.filterwarnings('ignore')\n",
    "\n",
    "from energis.run.orchestrator import run_all\n",
    "\n",
    "print(\"✅ Setup erfolgreich!\")\n",
    "print(f\"📁 Working Directory: {project_root}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1️⃣ Konfiguration laden"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Config-Pfade definieren\n",
    "cfg_paths = [\n",
    "    \"configs/base.yaml\",\n",
    "    \"configs/tech_catalog.yaml\",\n",
    "    \"configs/sites/default.site.yaml\",\n",
    "    \"configs/systems/baseline.system.yaml\",\n",
    "    \"configs/scenarios/pf_then_rh.scenario.yaml\",\n",
    "]\n",
    "\n",
    "# Optional: Overrides für Quick-Tests\n",
    "overrides = {\n",
    "    # \"run\": {\"solver\": \"glpk\"},  # Falls kein Gurobi\n",
    "    # \"site\": {\"year_target\": 2023}\n",
    "}\n",
    "\n",
    "print(\"📋 Konfiguration:\")\n",
    "for p in cfg_paths:\n",
    "    print(f\"  - {p}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2️⃣ Optimierung ausführen"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "%%time\n",
    "# Hauptlauf\n",
    "try:\n",
    "    res = run_all(cfg_paths, overrides=overrides if overrides else None)\n",
    "    print(\"\\n\" + \"=\"*60)\n",
    "    print(\"✅ OPTIMIERUNG ERFOLGREICH\")\n",
    "    print(\"=\"*60)\n",
    "    print(f\"📊 Export: {res['scenario_xlsx']}\")\n",
    "    print(f\"💰 Kosten: {res['costs']}\")\nexcept Exception as e:\n",
    "    print(f\"\\n❌ FEHLER: {e}\")\n",
    "    import traceback\n",
    "    traceback.print_exc()\n",
    "    res = None"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3️⃣ Ergebnisse laden & visualisieren"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "if res:\n",
    "    # Excel laden\n",
    "    ts = pd.read_excel(res['scenario_xlsx'], sheet_name='timeseries', index_col=0)\n",
    "    costs_df = pd.read_excel(res['scenario_xlsx'], sheet_name='costs', index_col=0)\n",
    "    \n",
    "    print(\"📈 Zeitreihen-Daten geladen:\")\n",
    "    print(f\"  Zeitschritte: {len(ts)}\")\n",
    "    print(f\"  Variablen: {ts.shape[1]}\")\n",
    "    print(f\"\\n🔑 Verfügbare Spalten:\")\n",
    "    for col in sorted(ts.columns):\n",
    "        print(f\"    - {col}\")\nelse:\n",
    "    print(\"⚠️  Keine Ergebnisse zum Laden verfügbar\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "if res:\n",
    "    # Plot 1: Strom-Bilanz\n",
    "    fig, axes = plt.subplots(2, 1, figsize=(14, 8))\n",
    "    \n",
    "    # Strom\n",
    "    ax = axes[0]\n",
    "    ts[['P_buy', 'P_sell']].plot(ax=ax, linewidth=1.5)\n",
    "    ax.set_title('⚡ Elektrische Leistung', fontsize=14, fontweight='bold')\n",
    "    ax.set_ylabel('MW')\n",
    "    ax.grid(alpha=0.3)\n",
    "    ax.legend(['Netzbezug', 'Einspeisung'])\n",
    "    \n",
    "    # Wärme\n",
    "    ax = axes[1]\n",
    "    heat_cols = [c for c in ts.columns if 'Q' in c or 'Qth' in c]\n",
    "    if heat_cols:\n",
    "        ts[heat_cols[:5]].plot(ax=ax, linewidth=1.5)  # Max 5 zur Übersicht\n",
    "    ax.set_title('🔥 Thermische Leistung (Top 5)', fontsize=14, fontweight='bold')\n",
    "    ax.set_ylabel('MW_th')\n",
    "    ax.set_xlabel('Zeit')\n",
    "    ax.grid(alpha=0.3)\n",
    "    \n",
    "    plt.tight_layout()\n",
    "    plt.show()\n",
    "    \n",
    "    # Speicher-SOC (falls vorhanden)\n",
    "    if 'TES_SOC' in ts.columns:\n",
    "        fig, ax = plt.subplots(figsize=(14, 4))\n",
    "        ts['TES_SOC'].plot(ax=ax, linewidth=2, color='orange')\n",
    "        ax.set_title('🔋 Thermischer Speicher - State of Charge', fontsize=14, fontweight='bold')\n",
    "        ax.set_ylabel('MWh')\n",
    "        ax.grid(alpha=0.3)\n",
    "        plt.tight_layout()\n",
    "        plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4️⃣ KPI-Zusammenfassung"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "if res:\n",
    "    print(\"\\n\" + \"=\"*60)\n",
    "    print(\"📊 KEY PERFORMANCE INDICATORS\")\n",
    "    print(\"=\"*60)\n",
    "    \n",
    "    # Kosten\n",
    "    print(f\"\\n💰 Gesamtkosten: {res['costs']['OBJ_value_EUR']:,.0f} EUR\")\n",
    "    print(f\"⚡ Peak-Leistung:  {res['costs']['P_buy_peak_MW']:.2f} MW\")\n",
    "    \n",
    "    # Energie-Mengen\n",
    "    print(f\"\\n📈 Energiebilanz:\")\n",
    "    print(f\"  Strombezug:    {ts['P_buy'].sum():.0f} MWh\")\n",
    "    print(f\"  Einspeisung:   {ts['P_sell'].sum():.0f} MWh\")\n",
    "    if 'Q_dump' in ts.columns:\n",
    "        print(f\"  Wärme-Dump:    {ts['Q_dump'].sum():.0f} MWh_th\")\n",
    "    \n",
    "    # Auslastung Heat Pumps\n",
    "    hp_cols = [c for c in ts.columns if c.startswith('HP') and c.endswith('_Q')]\n",
    "    if hp_cols:\n",
    "        print(f\"\\n🔥 Wärmepumpen-Auslastung:\")\n",
    "        for col in hp_cols:\n",
    "            avg_load = ts[col].mean()\n",
    "            max_load = ts[col].max()\n",
    "            print(f\"  {col:10s}: Ø {avg_load:5.1f} MW  (max {max_load:5.1f} MW)\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5️⃣ Export-Details"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "if res:\n",
    "    print(f\"\\n📁 Alle Ergebnisse gespeichert in:\")\n",
    "    print(f\"   {res['outdir']}\")\n",
    "    print(f\"\\n📄 Enthält:\")\n",
    "    print(f\"   - scenario.xlsx      (Zeitreihen + Kosten)\")\n",
    "    print(f\"   - merged_config.json (Vollständige Config)\")"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.11.0"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}

from energis.run.orchestrator import run_all
res = run_all()
res

NameError: name 'null' is not defined